# Basic App

Exploratory work around building the fish ID app. Mostly used as a scrap notebook, with most functionality moved to files once it's ready.

## Initial model

While there's no validation set, it seems likely that this model performs similarly to `rls-species-min-images-2` with `shorter-freeze`. The validation set was removed to maximise the training set and include species that have a single photo.

In [1]:
from fastai.vision.all import *
import experiments

In [2]:
learner = experiments.create_reproducible_learner(
    resnet18,
    Path("/home/yanir/projects/deep-fish/data/rls-species-m1-all/"),
    db_kwargs=dict(splitter=experiments.no_validation_splitter),
    learner_kwargs=dict(cbs=None),
)
learner.fine_tune(120, freeze_epochs=5)
learner.export("/home/yanir/projects/deep-fish/model.pkl")

epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.493670,None,None,None,None,00:40
1,8.913308,None,None,None,None,00:42
2,7.768785,None,None,None,None,00:45
3,6.509765,None,None,None,None,00:47
4,5.406819,None,None,None,None,00:47


/home/yanir/.conda/envs/deep-fish/lib/python3.9/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,4.135014,None,None,None,None,01:02
1,3.891555,None,None,None,None,01:03
2,3.647326,None,None,None,None,01:02
3,3.415774,None,None,None,None,01:03
4,3.181232,None,None,None,None,01:03
5,2.969606,None,None,None,None,01:03
6,2.786143,None,None,None,None,01:03
7,2.574696,None,None,None,None,01:03
8,2.363457,None,None,None,None,01:03
9,2.171068,None,None,None,None,01:03
